### Data Segmentation

In [9]:
import os
import random
import shutil

In [13]:
def move_random_files(src_dir, src_dir2, dest_dir, dest_dir2, k):
    # Get a list of all files in the source directory
    all_files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
    
    # If k is larger than the number of files, adjust k to the total number of files
    k = min(k, len(all_files))
    
    # Select k random files
    selected_files = random.sample(all_files, k)
    
    # Move each selected file to the destination directory
    for file_name in selected_files:
        src_path = os.path.join(src_dir, file_name)
        dest_path = os.path.join(dest_dir, file_name)
        shutil.move(src_path, dest_path)
        src_path = os.path.join(src_dir2, file_name.replace('.png', '.txt'))
        dest_path = os.path.join(dest_dir2, file_name.replace('.png', '.txt'))
        shutil.move(src_path, dest_path)

In [14]:
k = 150  # Number of files to move

move_random_files(r'datasets\train\images', r'datasets\train\labels', r'Augmented_Images\YOLO', r'Augmented_Images\YOLO', k)

### YOLO FORMATING

In [5]:
import xml.etree.ElementTree as ET # .XML parsing
import cv2 # Image processsing
import numpy as np
import os

In [21]:
# FUNCTION | LOAD IMAGES AND ANNOTATIONS #
def format_data_yolo(image_folder, annotation_folder, resized_image_folder, yolo_annotation_path, target_size=(224, 224)):

    # Iterate: Over all images
    for filename in os.listdir(image_folder):
        # Load image
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)

        original_height, original_width = img.shape[:2]  # Image original dimensions
        img_resized = cv2.resize(img, target_size)  # Resize the image

        # Save the resized image
        resized_img_path = os.path.join(resized_image_folder, filename)
        cv2.imwrite(resized_img_path, img_resized)
        
        # Load annotations
        annotation_file = os.path.join(annotation_folder, filename.replace('.png', '.xml'))
        tree = ET.parse(annotation_file)
        root = tree.getroot()

        height, width, _ = img_resized.shape
        yolo_annotations = []

        # Extract plate vertex coordinates
        for obj in root.findall('object'):
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # Resize the coordinates
            xmin = int(xmin * target_size[0] / original_width)
            ymin = int(ymin * target_size[1] / original_height)
            xmax = int(xmax * target_size[0] / original_width)
            ymax = int(ymax * target_size[1] / original_height)

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            # Assume a single class (car plate = 0)
            yolo_text = f"0 {x_center} {y_center} {box_width} {box_height}"

            if os.path.isfile(f'{yolo_annotation_path}\\{filename.replace(".png", ".xml")}'):
                os.remove(f'{yolo_annotation_path}\\{filename.replace(".png", ".xml")}')
            yolo_annotations.append(yolo_text)
            with open(f'{yolo_annotation_path}\\{filename.replace(".png", ".txt")}', "w") as f:
                f.write(yolo_text)

In [25]:
image_folder = r'datasets\train\images'
annotation_folder = r'datasets\train\labels'
resized_image_folder = r'datasets\train\images'
yolo_annotation_foledr = r'datasets\train\labels'
format_data_yolo(image_folder, annotation_folder, resized_image_folder, yolo_annotation_foledr)

### Data Augmentation

In [1]:
import os
import random
import numpy as np
from PIL import Image, ImageEnhance
import shutil
from lxml import etree


In [ ]:
# FUNCTION | ADD RANDOM NOISE #
def add_random_noise(image):
    np_image = np.array(image)
    noise = np.random.normal(0, 25, np_image.shape)
    noisy_image = np_image + noise
    noisy_image = np.clip(noisy_image, 0, 255)
    return Image.fromarray(np.uint8(noisy_image))

In [ ]:
# FUNCTION | ADD RANDOM COLOR JITTERING #
def apply_color_jitter(image):
    # Brightness
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.5, 1.5)) 

    # Contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(random.uniform(0.5, 1.5))

    # Saturation
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(random.uniform(0.5, 1.5))

    return image


In [18]:
# FUNCTION | AUGMENT AND SAVE IMAGE #
def augment_images(input_dir, output_dir, num_augments=5):

    image_files = [f for f in os.listdir(input_dir) if f.endswith(".png")]

    # Iterate: Over Images
    for image_file in image_files:
        # Base name
        base_name = os.path.splitext(image_file)[0]

        # Load the image
        image = Image.open(os.path.join(input_dir, image_file))

        # Randomly decide wether to augment image
        save = False
        if random.random() < 0.5:  # 50% / 50%
            augmented_image = add_random_noise(image)
            save = True
            if random.random() < 0.5:  # 50% / 50%
                augmented_image = apply_color_jitter(augmented_image)
        elif random.random() < 0.5:
            augmented_image = apply_color_jitter(image)
            save = True
        # Save augemented image
        if save:
            augmented_image.save(os.path.join(output_dir, f"{base_name}_augmented.png"))

In [20]:
input_directory = r'Augmented_Images\YOLO'
output_directory = r'Augmented_Images\YOLO\Augemented'

augment_images(input_directory, output_directory)


In [35]:
def get_png_filenames(directory):
    # List all files
    files = os.listdir(directory)
    
    # Keep .png files and remove extension and _augmented
    png_files = [os.path.splitext(f)[0].replace('_augmented', '') for f in files if f.endswith('.png')]
    
    return png_files

In [36]:
control_directory = r'Augmented_Images\Control\Augemented'
control_agumented_filenames = get_png_filenames(control_directory)

YOLO_directory = 'Augmented_Images\YOLO\Augemented'
YOLO_agumented_filenames = get_png_filenames(YOLO_directory)

In [37]:
for filename in control_agumented_filenames:
    shutil.copy(f'control\\train-valid\\labels\\{filename}.xml', f'Augmented_Images\\Control\\Labels\\{filename}_augmented.xml')

In [38]:
for filename in YOLO_agumented_filenames:
    shutil.copy(f'datasets\\train\\labels\\{filename}.txt', f'Augmented_Images\\YOLO\\Labels\\{filename}_augmented.txt')